In [1]:
# Obtiene una máscara para cada municipio del país.

!pip install bottleneck

import pandas as pd
import numpy as np

import xarray as xr

from multiprocessing import Pool

In [27]:
path_drive = "/content/drive/MyDrive/Colab/" 
path_data = path_drive + "data/"
f_mask = "municipios_mask.nc"
f_data = "data_0.csv"

# Abrimos los archivos a utilizar.
mask = xr.open_dataset( path_drive + f_mask )
df = pd.read_csv( path_data + f_data )

In [ ]:
# Procesa la variable para todos los años.
def proc(year):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d + var_2 + "/mexico_" 
        + var_2 + "." + str(year) + ".nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        df_y = df[ df["Año"] == year ].copy()
        for i in range( df["Municipio"].max() + 1 ):
            # Obtenemos el promedio dentro de la máscara
            # del municipio.
            df_y.loc[ ( df["Municipio"] == i ),
                var_1 ] = ( ds.where(mask
                .isel(municipio = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe().mean()[0] )
        return df_y

In [ ]:
# Variables de CHIRTS y CHIRPS.
vars = ["Tmax", "Tmin", "Tmean", "Pre"]

# Variable a procesar, 0 - 8.
n = 0
m = 0
var_1 =  df.columns[4:-9][n]
var_2 = vars[m]

if var_2 == "Pre":
    folder = "CHIRPS/"
else: folder = "CHIRTS/"

# Dirección de las variables.
path_d = path_drive + folder

# Años a procesar.
year = list(range(1995, 1996))

# Procesamos los años en paralelo.
pool = Pool()
df_r = pool.map(proc, year)

# Concatenamos los resultados de cada año.
for f in df_r:
    df.loc[df["Año"] == f["Año"].max() ] = f

# Guardamos el resultado como un nuevo archivo.
df.to_csv(path_data + "data_" + str(n + 1) + ".csv")